In [2]:
import calendar
import pandas as pd
import numpy as np

from django.db import models

from sister.modules.pembelajaran.models import Kelas
from sister.modules.presensi.models import PresensiSiswa

kelas = Kelas.objects.first()

class Manager:
    
    def get_siswa_kelas_df(self, kelas):
        siswa_kelas_df = pd.DataFrame(
            kelas.siswa.values('id', 'no_urut','siswa__person__full_name')
        )
        siswa_kelas_df['index'] = siswa_kelas_df['id']
        siswa_kelas_df = siswa_kelas_df.set_index('index')    
        siswa_kelas_df = siswa_kelas_df.rename(
            columns={
                'siswa__no_urut':'no',
                'siswa__person__full_name':'full_name'
            }
        )

        return siswa_kelas_df
    
    def get_presensi_siswa_df(self, kelas, semester, bulan):
        
        presensi = PresensiSiswa.objects.filter(
            presensi_kelas__kelas=kelas,
            presensi_kelas__semester=semester,
            presensi_kelas__tanggal__month=bulan
        ).annotate(
            tanggal=models.F('presensi_kelas__tanggal'),
            semester=models.F('presensi_kelas__semester'),
        ).values(
            'siswa_kelas_id', 'status', 'tanggal', 'semester'
        )

        presensi_df = pd.DataFrame(presensi)
        return presensi_df
        
    def get_rekap_presensi(self, kelas, semester, bulan):
        
        results_df = self.get_siswa_kelas_df(kelas)
        presensi_df = self.get_presensi_siswa_df(kelas, semester, bulan)

        days = calendar.monthrange(2020, bulan)[1]

        for day in range(1, days + 1):

            if not presensi_df.empty:
                result = presensi_df.loc[pd.to_datetime(presensi_df['tanggal']).dt.day == day]
                result = result[['siswa_kelas_id', 'status']]
                result = result.rename(columns={'status': day})
                result = result.set_index('siswa_kelas_id')
            else:
                result = results_df.copy()
                result[day] = np.nan
                result = result[day]

            results_df = results_df.join(
                result,
                on='id',
                how='left'
            )

        results_df = results_df.replace({np.nan: None})
        results_df['H'] = results_df.eq('H').sum(axis=1)
        results_df['S'] = results_df.eq('S').sum(axis=1)
        results_df['I'] = results_df.eq('I').sum(axis=1)
        results_df['A'] = results_df.eq('A').sum(axis=1)
        results_df['L'] = results_df.eq('L').sum(axis=1)
        return results_df

manager = Manager()
df = manager.get_rekap_presensi(kelas, 1, 7)
df

/home/sasri/Projects/sister/.venv/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/sasri/Projects/sister/.venv/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/sasri/Projects/sister/.venv/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/sasri/Projects/sister/.venv/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/hom

,id,no_urut,full_name,1,2,3,4,5,6,7,...,27,28,29,30,31,H,S,I,A,L
index,,,,,,,,,,,,,,,,,,,,,
6dc91e79-87ed-4ddf-8806-0e3568e3fec9,6dc91e79-87ed-4ddf-8806-0e3568e3fec9,1,Bacharudin Jusuf Habibie,None,None,None,None,None,None,None,...,None,None,None,None,None,0,0,0,0,0
